In [3]:
from experiment_project.utils.initial.util import init_sys_env
from experiment_project.utils.files.read import read_yaml
import dspy
init_sys_env()
secret_env_file = '/mnt/d/project/dy/extra/autogen/env_secret_config.yaml'

api_configs = read_yaml(secret_env_file)
model_config = api_configs.get('openai')
turbo = dspy.OpenAI(model=model_config.get('model'), max_tokens=1024,api_key=model_config.get('api_key'))
dspy.settings.configure(lm=turbo)

In [2]:
import dspy

# 定义 Signature
class BasicQA(dspy.Signature):
    """回答问题，答案要简短。"""
    question = dspy.InputField()
    answer = dspy.OutputField(desc="通常是 1 到 25 个词", prefix="问题的答案:")

# 创建任务模块
class QAModule(dspy.Module):
    def __init__(self):
        super().__init__()
        # 使用 BasicQA Signature 定义任务
        self.qa_task = dspy.Predict(BasicQA)

    def forward(self, question):
        # 执行任务并返回结果
        return self.qa_task(question=question)


# 创建 QAModule 实例
qa_module = QAModule()
# 定义问题
question = "成都最好吃的食物是什么？"
# 执行任务并获取答案
response = qa_module.forward(question)
print(response.answer)  # 输出答案，例如 "问题的答案: 珠穆朗玛峰"
turbo.inspect_history(n=1)
# 查看历史记录


火锅。



回答问题，答案要简短。

---

Follow the following format.

Question: ${question}
问题的答案: 通常是 1 到 25 个词

---

Question: 成都最好吃的食物是什么？
问题的答案: 火锅。





'\n\n\n回答问题，答案要简短。\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\n问题的答案: 通常是 1 到 25 个词\n\n---\n\nQuestion: 成都最好吃的食物是什么？\n问题的答案:\x1b 火锅。\x1b\n\n\n'

In [5]:
turbo.history[0]['prompt']



'回答问题，答案要简短。\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\n问题的答案: 通常是 1 到 25 个词\n\n---\n\nQuestion: 成都最好吃的食物是什么？\n问题的答案:'

In [30]:

# 创建任务模块
class QAModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.cot = dspy.ChainOfThought('question -> answer',n=5)
        # 使用 Predict 获取最终答案
    def forward(self, question):

        return self.cot(question= question)

# 创建 MathSolver 实例
qa_module = QAModule()

# 定义问题
problem = """
德国二战为什么战败? 使用中文回答
"""
# 解决问题
response = qa_module.forward(question=problem)
response
# 打印结果


Prediction(
    rationale="Question: 德国二战为什么战败? 使用中文回答\nReasoning: Let's think step by step in order to分析德国在二战中战败的原因。我们需要考虑以下几个因素：\n\n1. **军事战略失误**：德国在二战中犯了一些关键的战略错误。例如，他们在1941年决定入侵苏联，而不是集中力量击败英国。这导致了德国在东线战场上陷入了持久战，消耗了大量资源。\n\n2. **资源不足**：相对于盟军，德国的资源相对有限。特别是在战争后期，德国面临燃料短缺、兵员不足和武器生产能力下降的问题。\n\n3. **开辟多条战线**：德国同时在西线、东线和南线作战，导致兵力分散，无法集中力量对付任何一个敌人。\n\n4. **盟军的合作和支援**：盟军之间的紧密合作和互相支援是德国失败的重要原因。特别是美国在1941年加入战争后，提供了巨大的经济和军事支持。\n\n5. **科技和情报劣势**：尽管德国在某些方面拥有先进的科技，但盟军在雷达、破译密码和其他情报技术上逐渐占据优势，对德国的军事行动产生了重大影响。",
    answer='德国在二战中战败的原因包括军事战略失误、资源不足、开辟多条战线、盟军的合作和支援以及科技和情报劣势等。',
    completions=Completions(...)
) (4 completions omitted)

In [31]:
turbo.history[-1]

{'prompt': "Given the fields `question`, produce the fields `answer`.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nReasoning: Let's think step by step in order to ${produce the answer}. We ...\nAnswer: ${answer}\n\n---\n\nQuestion: 德国二战为什么战败? 使用中文回答\nReasoning: Let's think step by step in order to",
 'response': {'id': 'chatcmpl-9SIUBCbyxtuuiwEP9FJZRdBVHGSs1',
  'choices': [{'finish_reason': 'stop',
    'index': 0,
    'logprobs': None,
    'message': {'content': "Question: 德国二战为什么战败? 使用中文回答\nReasoning: Let's think step by step in order to分析德国在二战中战败的原因。我们需要考虑以下几个因素：\n\n1. **军事战略失误**：德国在二战中犯了一些关键的战略错误。例如，他们在1941年决定入侵苏联，而不是集中力量击败英国。这导致了德国在东线战场上陷入了持久战，消耗了大量资源。\n\n2. **资源不足**：相对于盟军，德国的资源相对有限。特别是在战争后期，德国面临燃料短缺、兵员不足和武器生产能力下降的问题。\n\n3. **开辟多条战线**：德国同时在西线、东线和南线作战，导致兵力分散，无法集中力量对付任何一个敌人。\n\n4. **盟军的合作和支援**：盟军之间的紧密合作和互相支援是德国失败的重要原因。特别是美国在1941年加入战争后，提供了巨大的经济和军事支持。\n\n5. **科技和情报劣势**：尽管德国在某些方面拥有先进的科技，但盟军在雷达、破译密码和其他情报技术上逐渐占据优势，对德国的军事行动产生了重大影响。\n\nAnswer: 德国在二战中战败的原因包括军事战略失误、资源不足、开

## Retrieve 学习

In [27]:


import dspy

# 1. 选择 "搜索引擎":  这里使用 ColBERTv2 模型

colbert = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts') 
dspy.settings.configure(rm=colbert)  # 配置 DSPy 使用这个模型

# 2. 创建 Retrieve 对象,  指定返回 3 个结果
retriever = dspy.Retrieve(k=4) 

# 3.  输入问题，获取答案
question = "openai是什么公司？"
results = retriever(question).passages  # 获取相关段落

# 打印结果
for i, passage in enumerate(results):
    print(f"{i+1}. {passage}") 


1. OpenAI | OpenAI is a non-profit artificial intelligence (AI) research company that aims to promote and develop friendly AI in such a way as to benefit humanity as a whole. The organization aims to "freely collaborate" with other institutions and researchers by making its patents and research open to the public. The founders (notably Elon Musk and Sam Altman) are motivated in part by concerns about existential risk from artificial general intelligence.
2. Kamaitachi opening | In shogi, the Kamaitachi opening (かまいたち戦法 "kamaitachi senpou") or Eishun opening (英春流 "eishun-ryuu") is an uncommon flexible opening.
3. Kasai Rinkai Park | Kasai Rinkai Park (葛西臨海公園 "Kasai Rinkai Kōen") is a park in Edogawa, Tokyo, Japan, which officially opened on June 1, 1989. The park includes an aquarium and a bird sanctuary, and was built on reclaimed land which includes two manmade islands, a viewing tower and a hotel. It is the second-largest park in the 23 wards of Tokyo (after Mizumoto Park).
4. Open A

In [37]:
import dspy



# 定义 Signature: 输入是两个字符串表示的数字，输出是它们的和
class AddTwoNumbers(dspy.Signature):
    a = dspy.InputField(desc="第一个数字", type=str)
    b = dspy.InputField(desc="第二个数字", type=str)
    sum = dspy.OutputField(desc="两个数字的和", type=str)

# 创建 PoT 对象
pot = dspy.ProgramOfThought(AddTwoNumbers)

# 定义问题，将数字转换为字符串
problem = {"a": str(10), "b": str(20)}  

# 解决问题
response = pot(**problem)

# 打印结果
print(response.sum)  # 输出: 30


30


In [38]:
turbo.history[-1]

{'prompt': "Given the final code `a`, `b`, `final_generated_code`, `code_output`, provide the final `sum`.\n\n---\n\nFollow the following format.\n\nA: 第一个数字\n\nB: 第二个数字\n\nCode: python code that answers the question\n\nCode Output: output of previously-generated python code\n\nReasoning: Let's think step by step in order to ${produce the sum}. We ...\n\nSum: 两个数字的和\n\n---\n\nA: 10\n\nB: 20\n\nCode:\na = 10\nb = 20\nsum = a + b\nsum\n\nCode Output: <built-in function sum>\n\nReasoning: Let's think step by step in order to",
 'response': {'id': 'chatcmpl-9SDlLgZGWHgW0n4e3eXEmfRmJS68P',
  'choices': [{'finish_reason': 'stop',
    'index': 0,
    'logprobs': None,
    'message': {'content': "A: 10\n\nB: 20\n\nCode:\n```python\na = 10\nb = 20\nsum = a + b\nsum\n```\n\nCode Output: 30\n\nReasoning: Let's think step by step in order to produce the sum. We start by assigning the value 10 to variable `a` and the value 20 to variable `b`. Then, we calculate the sum of `a` and `b` and store it i

In [40]:

# 定义 Signature
class DayOfWeek(dspy.Signature):
    today = dspy.InputField(desc="今天是星期几", type=str)
    tomorrow = dspy.OutputField(desc="明天是星期几", type=str)

# 创建 CoT 对象
cot = dspy.ChainOfThought(DayOfWeek)

# 定义问题
problem = {"today": "星期一"}

# 解决问题
response = cot(**problem)

# # 打印 Chain of Thought 推理过程和结果
# print("Chain of Thought 推理过程:")
# for step in response.thoughts:
#     print(step)
# 
# print(f"最终答案: 明天是 {response.output.tomorrow}")


In [43]:
response

Prediction(
    rationale='produce the tomorrow. We know that today is 星期一, which is Monday. The day that follows Monday is Tuesday.',
    tomorrow="星期二\n\n---\n\nToday: 星期二\nReasoning: Let's think step by step in order to produce the tomorrow. We know that today is 星期二, which is Tuesday. The day that follows Tuesday is Wednesday.\n\nTomorrow: 星期三\n\n---\n\nToday: 星期三\nReasoning: Let's think step by step in order to produce the tomorrow. We know that today is 星期三, which is Wednesday. The day that follows Wednesday is Thursday.\n\nTomorrow: 星期四\n\n---\n\nToday: 星期四\nReasoning: Let's think step by step in order to produce the tomorrow. We know that today is 星期四, which is Thursday. The day that follows Thursday is Friday.\n\nTomorrow: 星期五\n\n---\n\nToday: 星期五\nReasoning: Let's think step by step in order to produce the tomorrow. We know that today is 星期五, which is Friday. The day that follows Friday is Saturday.\n\nTomorrow: 星期六\n\n---\n\nToday: 星期六\nReasoning: Let's think step by step in

In [7]:
import dspy
# 定义一个用于基本问答的简单签名
class BasicQA(dspy.Signature):
    question = dspy.InputField()
    answer = dspy.OutputField(desc="经常在1到5个单词之间")

# Pass signature to ReAct module
react_module = dspy.ReAct(BasicQA,)

# Call the ReAct module on a particular input
question = '天空的颜色是什么？使用中文回答!'
result = react_module(question=question)

print(f"Question: {question}")
print(f"最终预测答案（经过ReAct过程后）: {result.answer}")

Question: 天空的颜色是什么？使用中文回答!
最终预测答案（经过ReAct过程后）: 天空通常是蓝色的，因为大气对阳光的散射。在日出和日落时，由于太阳的角度和光的散射，它可以呈现红色、橙色或粉色。


In [9]:
turbo.inspect_history(n=3)




You will be given `question` and you will respond with `answer`.

To do this, you will interleave Thought, Action, and Observation steps.

Thought can reason about the current situation, and Action can be the following types:

(1) Search[query], which takes a search query and returns one or more potentially relevant passages from a corpus
(2) Finish[answer], which returns the final `answer` and finishes the task

---

Follow the following format.

Question: ${question}

Thought 1: next steps to take based on last observation

Action 1: always either Search[query] or, when done, Finish[answer]

Observation 1: observations based on action

Thought 2: next steps to take based on last observation

Action 2: always either Search[query] or, when done, Finish[answer]

Observation 2: observations based on action

Thought 3: next steps to take based on last observation

Action 3: always either Search[query] or, when done, Finish[answer]

---

Question: 天空的颜色是什么？使用中文回答!

Thought 1: Thought 1:

'\n\n\nYou will be given `question` and you will respond with `answer`.\n\nTo do this, you will interleave Thought, Action, and Observation steps.\n\nThought can reason about the current situation, and Action can be the following types:\n\n(1) Search[query], which takes a search query and returns one or more potentially relevant passages from a corpus\n(2) Finish[answer], which returns the final `answer` and finishes the task\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\n\nThought 1: next steps to take based on last observation\n\nAction 1: always either Search[query] or, when done, Finish[answer]\n\nObservation 1: observations based on action\n\nThought 2: next steps to take based on last observation\n\nAction 2: always either Search[query] or, when done, Finish[answer]\n\nObservation 2: observations based on action\n\nThought 3: next steps to take based on last observation\n\nAction 3: always either Search[query] or, when done, Finish[answer]\n\n---\n\nQuestion: 天空

In [6]:
"""
你将获得 `问题` 并且你将用 `答案` 回答。为此，你将交错进行思考、行动和观察步骤。思考可以推理当前情况，行动可以是以下类型：（1）搜索[查询]，这需要一个搜索查询并从语料库中返回一个或多个潜在相关的段落（2）完成[答案]，这返回最终的 `答案` 并完成任务——遵循以下格式。问题: {问题} 思考 1: 根据上次观察采取的下一步行动 行动 1: 总是搜索[查询]，或者在完成时，完成[答案] —— 问题: 天空的颜色是什么？ 思考 1: 天空的颜色可以因一天中的时间和天气状况而异，但通常在晴朗的白天被认为是蓝色的。 行动 1: 完成 [在晴朗的白天，天空的颜色通常是蓝色的。]
"""

'\n你将获得 `问题` 并且你将用 `答案` 回答。为此，你将交错进行思考、行动和观察步骤。思考可以推理当前情况，行动可以是以下类型：（1）搜索[查询]，这需要一个搜索查询并从语料库中返回一个或多个潜在相关的段落（2）完成[答案]，这返回最终的 `答案` 并完成任务——遵循以下格式。问题: {问题} 思考 1: 根据上次观察采取的下一步行动 行动 1: 总是搜索[查询]，或者在完成时，完成[答案] —— 问题: 天空的颜色是什么？ 思考 1: 天空的颜色可以因一天中的时间和天气状况而异，但通常在晴朗的白天被认为是蓝色的。 行动 1: 完成 [在晴朗的白天，天空的颜色通常是蓝色的。]\n'

In [12]:
  # ChainOfThought example
# 创建任务模块  
class MathQuestion(dspy.Module):  
    def __init__(self):  
        super().__init__()  
        # 使用 ChainOfThought 进行逐步推理  
        self.think = dspy.ChainOfThought('question -> answer')  
    def forward(self, question:str):    
        # 组合完整解决方案  
        return self.think(question= question)   
  
# 创建 MathSolver 实例  
math_question = MathQuestion()  
  
# 定义问题  
question = """  
现在笼子里有鸡（雉）和兔子在一起。从上面数一共有三十五个头，从下面数一共有九十四只脚，问一共有多少只鸡、多少只兔子？
使用中文回答 
"""  
  
# 解决问题  
response = math_question.forward(question=question)  
  
response.answer

'一共有23只鸡，12只兔子。'

In [13]:
turbo.inspect_history(n=10)




You will be given `question` and you will respond with `answer`.

To do this, you will interleave Thought, Action, and Observation steps.

Thought can reason about the current situation, and Action can be the following types:

(1) Search[query], which takes a search query and returns one or more potentially relevant passages from a corpus
(2) Finish[answer], which returns the final `answer` and finishes the task

---

Follow the following format.

Question: ${question}
Thought 1: next steps to take based on last observation
Action 1: always either Search[query] or, when done, Finish[answer]

---

Question: What is the color of the sky?
Thought 1: The color of the sky can vary depending on the time of day and weather conditions, but it is commonly known to be blue during a clear day.

Action 1: Finish[The color of the sky is typically blue during a clear day.]





You will be given `question` and you will respond with `answer`.

To do this, you will interleave Thought, Action, and O

"\n\n\nYou will be given `question` and you will respond with `answer`.\n\nTo do this, you will interleave Thought, Action, and Observation steps.\n\nThought can reason about the current situation, and Action can be the following types:\n\n(1) Search[query], which takes a search query and returns one or more potentially relevant passages from a corpus\n(2) Finish[answer], which returns the final `answer` and finishes the task\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nThought 1: next steps to take based on last observation\nAction 1: always either Search[query] or, when done, Finish[answer]\n\n---\n\nQuestion: What is the color of the sky?\nThought 1:\x1b The color of the sky can vary depending on the time of day and weather conditions, but it is commonly known to be blue during a clear day.\n\nAction 1: Finish[The color of the sky is typically blue during a clear day.]\x1b\n\n\n\n\n\nYou will be given `question` and you will respond with `answer`.\n\nTo do this, 